<a href="https://colab.research.google.com/github/edermartelinho/Modelos_Linguagem_Neurais-LLM-/blob/main/M%C3%A9tricas_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Métricas de Avaliação para Modelos de Linguagem

Modelos de linguagem podem ser aplicados em diversas tarefas de processamento de linguagem natural. Para avaliar o desempenho de tais modelos e comparar suas performances nessas tarefas, diversas métricas são utilizadas. As métricas permitem quantificar objetivamente o desempenho na tarefa e fornece uma base comum de comparação. Além disso, análises mais detalhadas dos casos de erro mais extremos podem fornecer informações úteis para implementação de melhorias nas arquiteturas.

As métricas de avaliação necessitam também de um conjunto de dados para teste. O modelo em si, sem o fornecimento de dados de entrada e uma saída esperada, não fornece informação que permite julgar o seu desempenho, dessa forma conjuntos de dados são criados para a avaliação das tarefas. Como se fosse uma prova que aplicada para várias pessoas e corrigida pelo mesmo grupo de avaliadores, garante que a performance de todos seja comparável.

# Dependências

In [ ]:
!pip install datasets transformers evaluate sentencepiece sacremoses sacrebleu

In [ ]:
import transformers
transformers.logging.set_verbosity_error()

# Evaluator

In [ ]:
import evaluate
from datasets import load_dataset
from transformers import pipeline
from datasets import load_dataset
import random

## Perplexidade

As métricas intrínsicas avaliam o desempenho de modelos de linguagem em sua tarefa básica de treinamento, ou seja, capturar a distribuição estatística dos dados. Em modelos de linguagem auto-regressivo (GPT, LaMDA, PaLM) o modelamento estatístico busca definir a probabilidade do próximo token da sequência, atribuindo alta probabilidade aos tokens mais prováveis.

A medida de perplexidade exprime o nível de incerteza que o modelo possui ao definir a distribuição de probabilidade do próximo token dado uma sequência anterior. Assim, quanto maior a perplexidade mais incerto o modelo é sobre sua predição, distribuindo mais a probabilidade entre diversos tokens, ou seja, ao invés de ter somente um bom candidato para continuação da sequência são preditos múltiplos candidatos. É importante notar nessa definição que a perplexidade é somente uma medida de confiança do modelo quanto sua predição, mas **não** da sua acurácia. Formalmente a perplexidade é dada por:

$$
e^{- \frac{1}{N} \sum_{1}^{N} ln\, p_{\theta}(x_i|x_{i-1},...,x_{0})} = \prod_{1}^{N} p_{\theta}(x_i|x_{i-1},...,x_{0})^{-1/N}
$$

Onde $x_i$ é o token de maior probabilidade para sequência, $p_\theta$ é a probabilidade de um token dado uma sequência anterior e $N$ é o total de tokens na sequência.

Abaixo iremos testar três modelos com a mesma arquitetura, mas número de parâmetros diferentes. Para isso, utilizamos o dataset [IMDB]() que contem reviews de filmes e o módulo `evaluate` do Hugging Face para computar automaticamente a métrica.



In [ ]:
perplexity = evaluate.load("perplexity", module_type="metric")
dataset_texts = load_dataset("imdb", split="test").shuffle()["text"]

# Seleciona um subconjunto aleatório do dataset
n = len(dataset_texts)
subset = random.choices(range(n), k=500)
#dataset_texts = dataset_texts.select(subset)
dataset_texts = [dataset_texts[i] for i in subset]

results_70m = perplexity.compute(model_id='EleutherAI/pythia-70m-deduped',
                             predictions=dataset_texts,
                             batch_size=8)


results_160m = perplexity.compute(model_id='EleutherAI/pythia-160m-deduped',
                             predictions=dataset_texts,
                             batch_size=8)

results_410m = perplexity.compute(model_id='EleutherAI/pythia-410m-deduped',
                             predictions=dataset_texts,
                             batch_size=2)

Abaixo podemos ver os resultados de perplexidade para cada modelo.

In [ ]:
#print('Pythia 70M: ' + results_70m['mean_perplexity'])
#print('Pythia 160M: ' + results_160m['mean_perplexity'])
#print('Pythia 410M: ' + results_410m['mean_perplexity'])
print(f'Pythia 70M: {results_70m["mean_perplexity"]}')
print(f'Pythia 160M: {results_160m["mean_perplexity"]}')
print(f'Pythia 410M: {results_410m["mean_perplexity"]}')

## SQuAD V2

O Stanford Question Answering Dataset (SQuAD) é um conjunto de dados de triplas contendo um contexto, uma pergunta e uma resposta voltado para a avaliação da capacidade de compreensão de linguagem. Em cada tripla, a resposta é um trecho do texto de contexto que responde a pergunta. A tarefa do modelo é, a partir da sequência de entrada composta pelo contexto e pergunta, gerar a posição inicial e final do trecho do contexto que responde a pergunta. Adicionalmente, no SQuAD v2 existem perguntas que não podem ser respondidas a partir do contexto fornecido e o modelo deve ser capaz de indicar que não há resposta para pergunta.

O desempenho do modelo no dataset é medido com a métrica F1, a qual indica de maneira geral a qualidade do modelo em acertar o trecho da resposta. Pode-se também analisar a métrica F1 separadamente para os dados que possuem resposta e os que não podem ser respondidos.

In [ ]:
qna_model_tiny = 'deepset/tinyroberta-squad2'
qna_model_base = 'deepset/roberta-base-squad2'
qna_model_large = 'deepset/roberta-large-squad2'

squad_v2_dataset = load_dataset('squad_v2', split='validation')

# Seleciona um subconjunto aleatório do dataset
n = len(squad_v2_dataset)
subset = random.choices(range(n), k=1000)
squad_v2_dataset = squad_v2_dataset.select(subset)

In [ ]:
qna_eval = evaluate.evaluator('question-answering')

results_tiny = qna_eval.compute(model_or_pipeline=qna_model_tiny,
             data=squad_v2_dataset,
             squad_v2_format=True,
             metric='squad_v2')

results_base = qna_eval.compute(model_or_pipeline=qna_model_base,
             data=squad_v2_dataset,
             squad_v2_format=True,
             metric='squad_v2')

results_large = qna_eval.compute(model_or_pipeline=qna_model_large,
             data=squad_v2_dataset,
             squad_v2_format=True,
             metric='squad_v2')

In [ ]:
print(results_tiny['f1'], results_tiny['HasAns_f1'], results_tiny['NoAns_f1'])
print(results_base['f1'], results_base['HasAns_f1'], results_base['NoAns_f1'])
print(results_large['f1'], results_large['HasAns_f1'], results_large['NoAns_f1'])

## BLEU

Medir o desempenho de modelos de tradução é um processo complexo, pois envolve a avaliação de aspectos subjetivos como compreensão, fidelidade e fluência da tradução dado que não existe uma única tradução possível para um texto. A métrica BLEU é uma forma automatizada de comparar a tradução gerada por um modelo de linguagem com uma ou, preferencialmente, múltiplas traduções de referência feitas por profissionais. O valor BLEU exprime o quão próximo a tradução gerada pelo modelo está das traduções de referência. Dessa forma, quanto maior o seu valor, melhor a tradução. É importante destacar que atingir o valor máximo (100) de pontuação é extremamente improvável, pois iria requerer que todas as traduções fossem idênticas a uma referência. Além disso, ao utilizar poucas referências de tradução para comparação o resultado tende a ter valores menores

In [ ]:
en_pt_translation_dataset = load_dataset('opus_books', 'en-pt', split='train')

# Seleciona um subconjunto aleatório do dataset
n = len(en_pt_translation_dataset)
subset = random.choices(range(n), k=500)
en_pt_translation_dataset = en_pt_translation_dataset.select(subset)

english_texts = [translation_pair['en']
                 for translation_pair in en_pt_translation_dataset['translation']]

portuguese_texts = [translation_pair['pt']
                 for translation_pair in en_pt_translation_dataset['translation']]

en_pt_translation_dataset = en_pt_translation_dataset.add_column('english', english_texts)
en_pt_translation_dataset = en_pt_translation_dataset.add_column('portuguese', portuguese_texts)

In [ ]:
translation_model_opus = 'Helsinki-NLP/opus-mt-tc-big-en-pt'

sacrebleu_eval = evaluate.evaluator('translation')

results_opus = sacrebleu_eval.compute(model_or_pipeline=translation_model_opus,
             data=en_pt_translation_dataset,
             input_column='english',
             label_column='portuguese',
             metric='sacrebleu')

In [ ]:
print(results_opus['score'])